In [ ]:
import numpy as np
import pandas as pd
import pickle
from collections import Counter

#looking at relationships between companies trends
#can be used because relationships tend to lag behind, wait for first mover and then buy correlated companies
#measuring in percent change relationship over a week to norm things out a bit
def process_data(ticker):
    con_days = 7
    #create a join csv file with close data for certain amount
    #of days for all of the sp500
    df = pd.read_csv('sp500_closes.csv', index_col = 0)
    tickers = df.columns.values.tolist()
    df.fillna(0,inplace = True)
    
    for i in range(1, con_days+1):
        #this shifts price to look at something in the future
        #comparing todays price of mcdonalds with the next days(up until con_days) data
        df['{}_{}d'.format(ticker, i)] = (df[ticker].shift(-i) - df[ticker]) / df[ticker]
        df.fillna(0, inplace = True)
        return tickers, df
    
def b_or_s(*args):
    #if ANY of the days in the future (7 in this case) have a correlation this will return a value accordingly,
    #only one value is getting mapped to the ticker target in the func below
    cols = [c for c in args]
    trigger = 0.03 #in percent change
    for col in cols:
        if col > trigger:
            return 1
        elif col < -trigger: 
            return -1
    #this is ultimately generating features for training, a binary in a way 
    return 0
    
def extract_labels(ticker):
    tickers, df = process_data(ticker)
    df['{}_target'.format(ticker)] = list(map( b_or_s, 
                                             df['{}_1d'.format(ticker)],
                                             df['{}_2d'.format(tickeri)],
                                             df['{}_3d'.format(tickeri)],
                                             df['{}_4d'.format(ticker)],
                                             df['{}_5d'.format(ticker)],
                                             df['{}_6d'.format(ticker)],
                                             df['{}_7d'.format(ticker)]))
    
    vals = df['{}_target'.format(ticker)].values.tolist()
    str_vals = [str(i) for i in vals]
    print('Data Spread:', Counter(str_vals))
    df.fillna(0, inplace = True) #these are just to keep everything nice and tidy
    #if this isnt something better than print something not all buys its kinda useless
    #if this is too heavily buy, the strat is just buy and hold which is not the goal of this
    df = df.replace([np.inf, -np.inf], np.nan)
    df.dropna(inplace = True)
    df_feature_sets = df[[ticker_x for ticker_x in tickers]].pct_change() #pct change of 1 day
    df_feature_sets = df_feature_sets.replace([np.inf, -np.inf], 0)
    df_feature_sets.fillna(0, inplace = True)
    X = df_feature_sets.values
    y = df['{}_target'.format(ticker)].values
    return X, y, df
    #return feature sets labels and dataframes
    
    #X is feature sets and y is the lbales(output we want)
    
#with this kind of set up ready to run some ML things on it, some of this like data processing can be useful even outside of this specific strat that prob wont work but is to get a framework started
#replace b_or_s with something more complicated, rn our labels (1,-1,0) are only based on correlation between stocks within a certain date range
#can implement cool things with MyNN - this is a ML library I'm kind of working on